# Task 1

In this task, we will be analysing a dataset where each row corresponds to a clinical trial. There is a variety of information about the clinical trial given, such as the trial name, the conditions being monitored, and the start and completion times of the trial. The dataset contains around 500,000 rows, so Spark SQL is a good module for data processing, as this can be considered Big Data.

There are four questions relating to the dataset which we want to find the answers to, and I will write SQL queries in order to find this information. 

Our goals are to:
- Determine the frequency of each study type.
- Identify the top 10 most common conditions.
- Compute the average trial duration in months.
- Visualise trends in diabetes‑related trials over time.

By revealing these insights, we aim to help researchers and decision‑makers better understand trial designs, durations, and focus areas.

Firstly, we will upload the dataset and perform some exploratory data analysis, before tackling these questions with tables and visualisations.

#### Data Import

First, we upload our clinical trial CSV to DBFS and create a Spark SQL temporary view named `ClinicalTrials`.  
This step ensures that Spark can efficiently query the ~500 000‑row dataset using SQL.  
We specify `header = true` so the first row becomes column names, and `inferSchema = true` so Spark assigns the correct data types automatically.

In [0]:
%sql
-- Create a temporary view to load our clinical trial CSV data.
CREATE OR REPLACE TEMPORARY VIEW ClinicalTrials
USING csv
OPTIONS (
  path 'dbfs:/FileStore/tables/Clinicaltrial_16012025.csv',
  header 'true',
  inferSchema 'true'
);

#### Exploratory Data Analysis

Before we attempt to answer the four questions and reveal insights about the dataset, it is important to have an understanding of the data. This includes checking for potential issues that may arise, as well as confirming our initial assumptions.

1. **Schema Inspection**

We run `DESCRIBE TABLE ClinicalTrials` to verify all column names and data types. This allows us to create queries that use the correct attributes.

2. **Row Preview**  

By selecting the first 10 rows (`SELECT * FROM ClinicalTrials LIMIT 10`), we can look at actual examples of clinical trials. This will give us an idea of the values in key columns such as `Start Date` and `Conditions`, and can help us spot any anomalies in the data.

3. **Missing‑Value Analysis**

We count nulls in critical columns - `Start Date`, `Completion Date`, `Conditions`, and `Study Type` - to assess completeness.

4. **Distinct Value Cardinality**  

Counting distinct `Study Type` entries tells us how many unique categories exist, which informs us whether GROUP BY is appropriate for this attribute.

These checks establish confidence in the reliability of the dataset, and allow us to move forward for the analysis section.

In [0]:
%sql
DESCRIBE TABLE ClinicalTrials;

col_name,data_type,comment
NCT Number,string,null
Study Title,string,null
Acronym,string,null
Study Status,string,null
Conditions,string,null
Interventions,string,null
Sponsor,string,null
Collaborators,string,null
Enrollment,string,null
Funder Type,string,null


In [0]:
%sql
SELECT * 
FROM ClinicalTrials 
LIMIT 10;

NCT Number,Study Title,Acronym,Study Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Study Type,Study Design,Start Date,Completion Date
NCT05013879,Kinesiotape for Edema After Bilateral Total Knee Arthroplasty,null,COMPLETED,"Arthroplasty Complications|Arthroplasty, Replacement, Knee",DEVICE: Kinesio(R)Tape for edema control,Montefiore Medical Center,Burke Rehabilitation Hospital,65,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: SINGLE_GROUP|Masking: NONE|Primary Purpose: TREATMENT,2021-10-18,2023-11-24
NCT00517179,Effect of Vardenafil on Blood Pressure in Patients With Erectile Dysfunction Who Received Concomitant Doxazosin GITS,null,COMPLETED,Prostatic Hyperplasia|Impotence,DRUG: Vardenafil 10mg,"Hospital Authority, Hong Kong",null,40,OTHER_GOV,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CROSSOVER|Masking: DOUBLE|Primary Purpose: TREATMENT,2006-04,2007-05
NCT06714279,Laparoscopic-Assisted Transversus Abdominus Plane Block Versus Intraperitoneal Irrigation of Local Anesthetic for Patients Undergoing Laparoscopic Cholecystectomy,null,NOT_YET_RECRUITING,Laparoscopic Cholecystectomy|TAP Block|Local Anesthetic,DRUG: Tap Block - Bupivacaine|DRUG: Intraperitoneal infiltration to liver,"Royal College of Surgeons, Ireland",null,144,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: NONE|Primary Purpose: TREATMENT,2025-01,2025-01
NCT05600179,OCTA in Epivascular Glia After Dex Implant,null,COMPLETED,Diabetic Retinopathy,DRUG: Dexamethasone intravitreal implant,Federico II University,null,38,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2021-01-01,2022-09-30
NCT01511679,Brain-imaging and Adolescent Neuroscience Consortium,BANC,WITHDRAWN,Alcohol Abuse,null,Boston Children's Hospital,Massachusetts General Hospital|Mclean Hospital|Massachusetts Institute of Technology,0,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2012-09,2017-09
NCT05602779,Leverage Noninvasive Transcutaneous Vagus Nerve Stimulation to Reduce Suicidal Behaviors in Vulnerable Adolescents,null,RECRUITING,Self Harm|Suicidal Ideation,DEVICE: tVns Program|OTHER: Phone App Program|COMBINATION_PRODUCT: tVNS and Phone App Program|OTHER: Enhanced Treatment as Usual,University of Notre Dame,University of Rochester,212,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: SINGLE (PARTICIPANT)|Primary Purpose: PREVENTION,2023-10-08,2027-09-30
NCT04175379,The Effect of Permissive Hypercapnia on Oxygenation and Post-operative Pulmonary Complication During One-lung Ventilation,null,UNKNOWN,Thoracic Surgery,OTHER: group 40|OTHER: group 50|OTHER: group 60,Yonsei University,null,279,OTHER,INTERVENTIONAL,"Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: TRIPLE (PARTICIPANT, CARE_PROVIDER, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",2019-11-25,2021-10
NCT01126879,Genistein in Treating Patients With Prostate Cancer,null,TERMINATED,Adenocarcinoma of the Prostate|Recurrent Prostate Cancer|Stage I Prostate Cancer|Stage II Prostate Cancer|Stage III Prostate Cancer,DIETARY_SUPPLEMENT: genistein|OTHER: placebo|PROCEDURE: therapeutic conventional surgery,Northwestern University,National Cancer Institute (NCI),12,OTHER,INTERVENTIONAL,"Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (PARTICIPANT, INVESTIGATOR)|Primary Purpose: TREATMENT",2011-02-03,2013-12-28
NCT03058679,Trial of Specific Carbohydrate and Mediterranean Diets to Induce Remission of Crohn's Disease,DINE-CD,COMPLETED,Crohn Disease,OTHER: Diet,University of Pennsylvania,"Patient-Centered Outcomes Research Institute|Crohn's and Colitis Foundation|University of North Carolina, Chapel Hill",197,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: NONE|Primary Purpose: TREATMENT,2017-09-29,2020-03-01
NCT05531279,A Study of PEG-rhG-CSF and rhG-CSF Used for Aplastic Anemia Granulocyte Deficiency,null,RECRUITING,Severe Aplastic Anemi

In [0]:
%sql
SELECT
  SUM(CASE WHEN `Start Date`      IS NULL THEN 1 ELSE 0 END) AS missing_start,
  SUM(CASE WHEN `Completion Date` IS NULL THEN 1 ELSE 0 END) AS missing_completion,
  SUM(CASE WHEN `Conditions`      IS NULL THEN 1 ELSE 0 END) AS missing_conditions,
  SUM(CASE WHEN `Study Type`      IS NULL THEN 1 ELSE 0 END) AS missing_study_type
FROM ClinicalTrials;

missing_start,missing_completion,missing_conditions,missing_study_type
5190,16672,953,919


In [0]:
%sql
SELECT
  COUNT(DISTINCT `Study Type`) AS num_study_types
FROM ClinicalTrials; 

num_study_types
115


#### EDA Findings

From the exploratory data analysis, we found some key insights:

- There are some problems with the schema. `Start Date` and `Completion Date` should be of the `timestamp` data_type, but the `DESCRIBE TABLE` function claims they are strings. Also, we can see from the Row Preview that `Enrollment` should be of the `int` data_type (this integer value probably refers to the number of people taking part in the trial), but this also says string. This makes me believe there is a problem with how the data was parsed.

- The `Conditions` column uses `|` as the sole delimiter, not commas or semicolons, so our split logic must target this symbol.

- Date strings appear in two formats - `yyyy-mm-dd` and `yyyy-mm` - so we should coalesce two `to_date()` calls to handle both.

- Approximately 3% of trials are missing completion dates, which we’ll filter out when calculating average durations.

- There are 115 distinct study types (including NULL), which seems large. Further analysis will have to be done on this to find anomalous data.

With this information in mind, we can now be more confident in answering the given questions. However, the first step is to fix the mis-parsed data, so that the dataset is aligned correctly. To do this, I will analyse the rows which have been mis-parsed and look for potential issues.

In [0]:
%sql
-- Find rows where `Enrollment` is not an integer
SELECT *
FROM ClinicalTrials
WHERE NOT Enrollment RLIKE '^[0-9]+$'
LIMIT 10;

NCT Number,Study Title,Acronym,Study Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Study Type,Study Design,Start Date,Completion Date
NCT02520479,"""""""Sandwich"""" Chemotherapy With Radiotherapy in Newly Diagnosed",Stage IE to IIE,"ENKTL""",null,TERMINATED,Treatment Refusal,DRUG: P-CHOP|RADIATION: Radiotherapy|DRUG: P-CHOP,Sun Yat-sen University,null,12,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROUP|Masking: NONE|Primary Purpose: TREATMENT
NCT06727279,Assessment of Tolerability of Specialized Food Products Made Out of Vegetable Protein and Their Influence on Lipid Profile in Patients with Non-alcoholic Fatty Liver Disease,null,NOT_YET_RECRUITING,Non-alcoholic Fatty Liver Disease NAFLD,OTHER: Specialized food - Plant-Based Meat Analog|OTHER: standard isocalorie diet,"""Federal State Budgetary Scientific Institution """"Federal Research Centre of Nutrition","Biotechnology""",Group of companies EFKO|Russian Science Foundation,50,OTHER,INTERVENTIONAL,"Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: TRIPLE (PARTICIPANT, CARE_PROVIDER, INVESTIGATOR)|Primary Purpose: BASIC_SCIENCE",2025-01-20
NCT04616664,"""Diagnostic Accuracy of the """"LLIFT""""","a Novel Non-invasive Biomarker for the Diagnosis of Non Alcoholic Fatty Liver (NAFL) and SteatoHepatitis (NASH) in a Population With High Risk of Metabolic Syndrome""""""",LLIFT,UNKNOWN,Obesity|NASH - Nonalcoholic Steatohepatitis,null,"University Hospital, Lille","RHU PreciNASH Task 1.4|Région Nord-Pas de Calais, France",1027,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2021-02-16
NCT02519764,"""Evaluation of the Impact of a Hydration Protocol """"at Thirst"""" on Natremia of the Ultra Trail du Mont-Blanc","2015 Runners""",NATRITRAIL,COMPLETED,Athletes|Healthy,OTHER: Hydration when thirsty.|OTHER: Not hydration when thirsty,Centre Hospitalier Universitaire de Nīmes,DOKEVER,198,OTHER,INTERVENTIONAL,Allocation: NON_RANDOMIZED|Intervention Model: PARALLEL|Masking: SINGLE (INVESTIGATOR)|Primary Purpose: BASIC_SCIENCE,2015-08
NCT02127164,Vacuum Assisted Therapy in Emergent Contaminated Abdominal Surgeries,null,COMPLETED,Gastrointestinal Injury|Complicated Diverticulitis,"""DEVICE: """"Veraflo"""" device","Dakin's solution""",University of Arizona,3M,18,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROUP|Masking: NONE|Primary Purpose: TREATMENT,2014-05
NCT04747990,coMpliAnce With evideNce-based cliniCal Guidelines in the managemenT of Acute biliaRy pancreAtitis,MANCTRA-1,COMPLETED,Acute Pancreatitis|Acute Pancreatic Necrosis|Acute Pancreatitis Due to Gallstones|Acute Pancreatic Fluid Collection|Acute Pancreatitis With Infected Necrosis|Acute Pancreatitis Without Necrosis or Infection|Acute Pancreatitis Due to Common Bile Duct Calculus|Acute Pancreatitis Recurrent|Acute Pancreatitis Without Necrosis or Infection (Diagnosis)|Covid19,PROCEDURE: Early Definitive Treatment,University of Cagliari,"""Chiara Gerardi - Istituto Di Ricerche Farmacologiche Mario Negri|Federico Coccolini - General, Emergency and Trauma Surgery, Pisa|Salomone di Saverio -Department of Surgery, Varese|Gianluca Pellino - Universitá degli Studi della Campania 'Luigi Vanvitelli', Naples|Francesco Pata - General Surgery Unit, Nicola Giannettasio Hospital, Corigliano-Rossano|Benedetto Ielpo - HPB Surgery Unit, Hospital del Mar, Barcelona|Francesco Virdis - Trauma and Acute Care Surgery Unit, Milan|Dimitrios Damaskos - Royal Infirmary of Edinburgh, Edinburgh|Stavros Gourgiotis - Addenbrooke's Hospital, Cambridge|Gaetano Poillucci - Department of Surgery """"Paride Stefanini""""",Rome|Daniela Pacella - University of Naples Federico II,Naples|Kumar Jayant- University of Chicago,USA|Ferdinando Agresta- Vittorio Veneto Civil Hospital,Italy|Ari Leppaniemi - University of Helsinki,Finland|Fausto Catena - Maggiore Hospital,Parma|Yoram Kluger - Rambam Health care campus
NCT00737490,Does Echocardiographically Guided Ventriculo-Ventricular 

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW raw_lines
USING text
OPTIONS (
  path 'dbfs:/FileStore/tables/Clinicaltrial_16012025.csv',
  header 'true',
  inferSchema 'true'
);

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW ClinicalTrials_Clean AS
SELECT
  regexp_replace(
    value,
    '(?<=\\d)"(?=\\d)',   
    ''                     -- removes any " symbol between digits
  ) AS value
FROM raw_lines;

In [0]:
%sql
INSERT OVERWRITE DIRECTORY 'dbfs:/FileStore/tables/Clinicaltrial_16012025_clean.csv'
USING text
SELECT value
FROM ClinicalTrials_Clean;


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW ClinicalTrials
USING CSV
OPTIONS (
  path               "dbfs:/FileStore/tables/Clinicaltrial_16012025_clean.csv",
  header             "true",
  inferSchema        "true",
  quote              "\"",
  escape             "\""
);


#### Data Preprocessing

1. Firstly, we identified any rows which did not have an integer value in the `Enrollment` attribute. This was done using a pure integer regex. The results confirmed that stray `"` characters were causing the problems, as these were found in cells before mis-parsing occurred.
2. Next, we loaded each line of the original CSV into a temporary view so that we could manipulate the original text.
3. Using a regex replace, we removed any examples of `"` which appeared between digits. This would keep the valid quotes throughout the text, and allow the CSV to be parsed properly.
4. We then wrote the cleaned text lines into a new directory via `INSERT OVERWRITE DIRECTORY`. This preserves the original structure of the CSV, while replacing the faulty records with cleaned records.
5. Finally, the new table `ClinicalTrials` was created with standard `header` and `inferSchema` settings. This would ensure all of the columns were of the correct data_type, as can be seen below, and that there were the correct amount of categories of `Study Type`.

In [0]:
%sql
DESCRIBE TABLE ClinicalTrials;

col_name,data_type,comment
NCT Number,string,null
Study Title,string,null
Acronym,string,null
Study Status,string,null
Conditions,string,null
Interventions,string,null
Sponsor,string,null
Collaborators,string,null
Enrollment,int,null
Funder Type,string,null


In [0]:
%sql
SELECT
  COUNT(DISTINCT `Study Type`) AS num_study_types
FROM ClinicalTrials;

num_study_types
3


#### Question 1: Frequency of Study Types

First, we determine how many trials fall into each study type to understand which designs are most common. We:

1. **Group** the data by the `Study Type` column.  
2. **Count** the number of trials for each type using `COUNT(*)`.  
3. **Order** the results in descending order so the most frequent types appear first.

This query reveals the distribution of the 4 different study types. There are 966 examples of `EXPANDED_ACCESS`, which are rarer than the usual two categories, meanwhile the 900 null rows correspond with trials with withheld study status and redacted information.



In [0]:
%sql
-- Question 1: List the three clinical trial types (Study Type) and their frequency, sorted descending.
SELECT
  `Study Type`   AS study_type,
  COUNT(*)       AS frequency
FROM ClinicalTrials
GROUP BY `Study Type`
ORDER BY frequency DESC;

study_type,frequency
INTERVENTIONAL,399883
OBSERVATIONAL,120904
EXPANDED_ACCESS,966
null,900


In [0]:
%sql
-- List rows where Study Type is NULL.
SELECT *
FROM ClinicalTrials
WHERE `Study Type` IS NULL
LIMIT 10;

NCT Number,Study Title,Acronym,Study Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Study Type,Study Design,Start Date,Completion Date
NCT01470079,[Trial of device that is not approved or cleared by the U.S. FDA],null,WITHHELD,null,null,[Redacted],null,null,null,null,null,null,null
NCT05674279,[Trial of device that is not approved or cleared by the U.S. FDA],null,WITHHELD,null,null,[Redacted],null,null,null,null,null,null,null
NCT01662479,[Trial of device that is not approved or cleared by the U.S. FDA],null,WITHHELD,null,null,[Redacted],null,null,null,null,null,null,null
NCT05395884,[Trial of device that is not approved or cleared by the U.S. FDA],null,WITHHELD,null,null,[Redacted],null,null,null,null,null,null,null
NCT04787484,[Trial of device that is not approved or cleared by the U.S. FDA],null,WITHHELD,null,null,[Redacted],null,null,null,null,null,null,null
NCT02474784,[Trial of device that is not approved or cleared by the U.S. FDA],null,WITHHELD,null,null,[Redacted],null,null,null,null,null,null,null
NCT03967184,[Trial of device that is not approved or cleared by the U.S. FDA],null,WITHHELD,null,null,[Redacted],null,null,null,null,null,null,null
NCT01008202,[Trial of device that is not approved or cleared by the U.S. FDA],null,WITHHELD,null,null,[Redacted],null,null,null,null,null,null,null
NCT05059002,[Trial of device that is not approved or cleared by the U.S. FDA],null,WITHHELD,null,null,[Redacted],null,null,null,null,null,null,null
NCT05694702,[Trial of device that is not approved or cleared by the U.S. FDA],null,WITHHELD,null,null,[Redacted],null,null,null,null,null,null,null


#### Question 2: Top 10 Conditions by Frequency

Next, we determine which conditions were most frequently tracked during the clinical trials. As more than one condition can be tracked per trial, and multiple values were found in the `Conditions` column per row, we must first “flatten” these lists so each condition appears as its own record. We found in the EDA stage that each condition is separated by the `|` delimiter. Therefore, we:

1. **Use** the `split(Conditions, '\\|')` function to break each string into an array of individual conditions.
2. **Normalise** the data using `explode()`, creating a new row for each element.
3. **Trim** each condition to remove potential whitespaces from causing duplicates in the data.
4. **Group by** the trimmed condition, and count all occurences.
5. **Limit** the results to show the top 10 entries in `DESC` order.

This gives a list of the ten most commonly investigated conditions across all clinical trials. 

In [0]:
%sql
-- Question 2: Extract and count individual conditions, splitting on '|'
SELECT
  trim(condition) AS condition,
  COUNT(*)        AS frequency
FROM (
  -- explode each Conditions string into multiple rows
  SELECT explode(split(Conditions, '\\|')) AS condition
  FROM ClinicalTrials
  WHERE Conditions IS NOT NULL
) exploded
GROUP BY trim(condition)
ORDER BY frequency DESC
LIMIT 10;

condition,frequency
Healthy,10309
Breast Cancer,7941
Obesity,6954
Stroke,4484
Hypertension,4255
Depression,4196
Prostate Cancer,4071
Pain,4057
HIV Infections,3819
Cancer,3529


#### Question 3: Mean Clinical Trial Length in Months

Next, we calculate the average duration (in months) for studies which have both start and completion dates recorded. We learnt in the EDA stage that the date columns have two date formats: `yyyy-mm-dd` and `yyyy-mm`, so we use `coalesce` to ensure all the strings are parsed correctly. The important steps are as follows:

1. **Filter** out trials with NULL `Start Date` or `Completion Date`.
2. **Parse** dates using a combination of `coalesce` and `to_date`. The former function takes the non-null value of the two values given by the latter function, and the latter function will always provide at most one non-null value (if there are two non-null values, the row is discarded due to the first step).
3. **Compute** the difference in months with the `months_between` function.
4. **Obtain** the mean length using `AVG()`.

The mean clinical trial length in months is around 35.6, and this provides a robust measure of the average trial.

In [0]:
%sql
-- Question 3: Calculate the mean clinical trial length (in months),
-- handling both full dates (yyyy-MM-dd) and year-month only (yyyy-MM).
SELECT
  AVG(
    months_between(
      -- Either yields full date or month date
      coalesce(
        to_date(`Completion Date`, 'yyyy-MM-dd'),
        to_date(`Completion Date`, 'yyyy-MM')
      ),
      coalesce(
        to_date(`Start Date`,      'yyyy-MM-dd'),
        to_date(`Start Date`,      'yyyy-MM')
      )
    )
  ) AS avg_trial_length_months
FROM ClinicalTrials
WHERE `Start Date`      IS NOT NULL
  AND `Completion Date` IS NOT NULL;

avg_trial_length_months
35.571426600558915


#### Question 4: Diabetes-Related Completed Trials Over Time

For the final question, we visualise the number of completed clinical trials for diabetes over time. We use `Completion Date` to mark when the trial is counted in our query. The steps to achieve this visualisation are:

1. **Filter** trials where:
   - `Completion Date` is not null
   - `Study Status` is ‘Completed’
   - `Conditions` contains ‘diabetes’.
2. **Coalesce** the dates in `Completion Date` as we did in Question 3.
3. **Extract** each year from the dates using the `year()` function.
4. **Group by** year, this counts how many trials happened each year.
5. **Order** results chronologically.
6. **Choose** an appropriate visualisation. 

I went with a line chart, as this is best practice for showing how a variable changes over time.

The graph shows that diabetes trials were less common in the 1990s, before blooming in popularity in the 2000s and 2010s. This provides insight into attitudes surrounding diabetes.


In [0]:
## Question 4: Count diabetes-related completed studies grouped by completion year.
trend_df = spark.sql("""
SELECT
  year(
    coalesce(
      to_date(`Completion Date`, 'yyyy-MM-dd'),
      to_date(`Completion Date`, 'yyyy-MM')
    )
  ) AS completion_year,
  COUNT(*) AS diabetes_trial_count
FROM ClinicalTrials
WHERE `Completion Date` IS NOT NULL
  AND lower(`Conditions`) LIKE '%diabetes%'
  AND lower(`Study Status`) = 'completed'
GROUP BY
  year(
    coalesce(
      to_date(`Completion Date`, 'yyyy-MM-dd'),
      to_date(`Completion Date`, 'yyyy-MM')
    )
  )
ORDER BY completion_year
""")

# Visualise the data
display(trend_df)


completion_year,diabetes_trial_count
1989,2
1990,1
1991,1
1992,3
1993,3
1994,2
1995,2
1996,2
1997,3
1998,10


Databricks visualization. Run in Databricks to view.

#### Conclusion

Through our Spark SQL analysis of this clinical trials dataset we have:

- Preprocessed the data such that the schema is correct - this aligned faulty columns.
- Conducted exploratory data analysis, finding key details to inform our decision making surrounding the questions
- Answered 4 questions with succinct queries, discovering important information about study types, top conditions, average trial duration and diabetes trial frequency.

These insights can advise many business practicalities, such as budget planning, trial prioritisation and strategic decision-making across various fields.
